<div style="color: red;">

# **2章　自然言語と単語の分散表現**

<div style="color: black; font-size: 20px">

これからいよいよ自然言語処理の世界へ足を踏み入れます。  
自然言語処理が扱う分野は多岐にわたりますが、  
その本質的問題はコンピュータに私たちの言葉を理解させることにあります。  

本章では、コンピュータに言葉を理解させるとはどういうことか、  
そしてどのようなアプローチが存在するのか、ということを中心に考察を進めます。  

特に古典的な手法 ――ディープラーニング登場以前の手法―― について詳しく見ていきます。  
そして次章から、ディープラーニングをベースとした手法へと進みます。  

また、本章ではテキストを Python で扱う練習も行います。  
テキストを単語に分割する処理や、単語を単語 ID に変換する処理などを実装します。  

そして本章で実装する関数は、次章以降でも利用します。  
つまり本章は、これから先のテキスト処理の下準備も兼ねています。  

それでは、自然言語処理の世界へと進みましょう！
</div>

<div style="color: red;">

## **2.1 自然言語処理とは**
</div>

<div style="color: black; font-size: 20px">

**自然言語処理**（Natural Language Processing：NLP）とは、文字どおり解釈すれば、「自然言語を処理する分野」。  
分かりやすく言うと、「私たちの言葉をコンピュータに理解させるための技術（分野）」です。  
つまり、自然言語処理の目標とすることは、人の話す言葉をコンピュータに理解させ、  
私たちにとって役に立つことをコンピュータに行わせることにあります。  

コンピュータが理解できる「プログラミング言語」や「マークアップ言語」は、  
コードの意味が一意に解釈できるように文法が定義されています。  
これは言ってみれば、**"固い言語"** と言えます。  

一方、英語や日本語などの自然言語は、**"柔らかい言語"** です。  
「柔らかい」というのは、同じ意味の文章にさまざまな表現があったり、文章に曖昧さがあったりと、意味や形が柔軟に変わることを意味します。  
また、時代とともに新しい言葉や新しい意味が生まれます（または、廃れます）。これも自然言語の "柔らかさ" の表れです。

頭の固いコンピュータに自然言語を理解させるという難問をクリアできれば、人にとって役に立つことをコンピュータに行わせることができます。  
たとえば、検索エンジンや機械翻訳、質問応答システムやIME（かな漢字変換）、文章の自動要約や感情分析など、  
私たちの身の回りではすでに自然言語処理の技術が数多く使われています。
</div>

<div style="color: black; font-size: 20px">

![代替テキスト](./kazuki.a_images/Section2/Hint/Hint2-1_1.png)
</div>

<div style="color: red;">

### **2.1.1 単語の意味**
</div>

<div style="color: black; font-size: 20px">

私たちの言葉の意味は「**単語**」によって構成されます。  
**単語**は、言ってみれば、**意味の最小単位**です。  
そのため、自然言語をコンピュータに理解させるためには、**「単語の意味」を理解させる**ことが重要であると言えそうです。

本章のテーマは、コンピュータに「単語の意味」を理解させることです。  
より正確に言うと、「単語の意味」をうまく捉えた表現方法について考えます。  
具体的には、本章と次章で次の３つの手法を見ていきます。

- シソーラスによる手法【本章】
- カウントベースの手法【本章】
- 推論ベースの手法（word2vec）【次章】

最初に、人の手によって作られた**シソーラス**（類語辞書）を利用する方法について簡単に見ていきます。  
続いて、統計情報から単語を表現する手法 ――ここでは「**カウントベースの手法**」と呼ぶことにします―― について説明します。  
ここまでが本章で学ぶ内容です。  

そして次章では、ニューラルネットワークによる「**推論ベース**」の手法を扱います。  
なお、本章の構成は、スタンフォード大学の授業「CS224d: Deep Learning for Natural Language Processing」[10] を参考にしています。
</div>

<div style="color: red;">

## **2.2 シソーラス**
</div>

<div style="color: black; font-size: 20px">

「単語の意味」を表すためには、人の手によって単語の意味を定義することが考えられます。  

そのひとつの方法として、『広辞苑』などの辞書のように、一つひとつの単語に対してその単語の意味を説明していくことが考えられます。  
たとえば、「自動車」という単語を辞書で引くと、「車輪を取り付けてそれによって進むようになっている乗り物や運搬具……」のような説明が続きます。  

そのように単語を定義すれば、コンピュータも単語の意味を理解できるかもしれません。  
自然言語処理の歴史を振り返ってみると、単語の意味を人手で定義するという試みは数多く行われてきました。  
ただし、『広辞苑』のように人が使う一般的な辞書ではなく、**シソーラス**（thesaurus）と呼ばれるタイプの辞書が多く使われてきました。

**シソーラス** とは（基本的には）類語辞書であり、「同じ意味の単語（同義語）」や「意味の似た単語（類義語）」が同じグループに分類されています。  
たとえば、`car` の同義語には、`automobile` や `motorcar` などが存在することが、シソーラスを使えば分かります（図2-1 参照）。

![代替テキスト](./kazuki.a_images/Section2/Figure/図2-1_同義語の例.png)
</div>


<div style="color: black; font-size: 20px">

また、自然言語処理において利用されるシソーラスでは、  
単語の間で「上位と下位」「全体と部分」など、より細かい関連性が定義されている場合があります。  

具体的には、図2-2 のように、グラフ構造によって各単語の関係性が定義されているケースです。
![代替テキスト](./kazuki.a_images/Section2/Figure/図2-2_単語ネットワーク.png)

図2-2 では、「car」という単語の上位概念として、「motor vehicle（動力車）」という単語が位置しています。  
また、「car」の下位の概念に、「SUV」「compact」「hatch-back」など、より具体的な車種があることが示されています。  

このようにすべての単語に対して、類義語の集合を作り、また、それぞれの単語の関係をグラフで表現することで、単語間のつながりを定義できます。  
私たちはこの "単語ネットワーク" を利用することで、コンピュータに単語間の関連性を教えることができます。

これは、コンピュータに単語の意味を（間接的にであれ）授けることができたと言えそうです。  
そして、その知識を利用すれば、私たちにとって有用なことをコンピュータに行わせることができるでしょう。
</div>

<div style="color: black; font-size: 20px">

![代替テキスト](./kazuki.a_images/Section2/Hint/Hint2-2_1.png)
</div>

<div style="color: red;">

### **2.2.1 WordNet**
</div>

<div style="color: black; font-size: 20px">

自然言語処理の分野において、最も有名なシソーラスは **WordNet** です。  

WordNet はプリンストン大学で1985年に開発がスタートした伝統あるシソーラスで、これまでに多くの研究で利用されてきました。  
また、さまざまな自然言語処理のアプリケーションにおいても大いに活躍しています。  

WordNet を使えば、類義語を取得したり、“単語ネットワーク” を利用したりすることができます。  
また、単語ネットワークを使って単語間の類似度を算出することも可能です。  

ここでは、WordNet について詳細な説明は行いません。  
WordNet を使ったPython の実装について興味のある方は、「付録B WordNet を動かす」を参照してください。  
付録B では、WordNet をインストールして（正確には NLTK というモジュールをインストールして）、いくつかの簡単な実験を行います。

![代替テキスト](./kazuki.a_images/Section2/Hint/Hint2-2_2.png)
</div>

<div style="color: red;">

### **2.2.1 シソーラスの問題点**
</div>

<div style="color: black; font-size: 20px">

WordNet のようなシソーラスでは、多くの単語に対して同義語や階層構造などの関係性が定義されています。  
そして、それらの知識を利用すれば、「単語の意味」というものを（間接的にであれ）コンピュータに授けることができそうです。  
しかし、それらの人の手によるラベル付けには次の大きな欠点が存在します。  
<br>

**時代の変化への対応が困難**  
私たちの使う言葉は生きています。時とともに新しい言葉が生まれ、古い言葉はいつの日か忘れ去られます。  
たとえば、「クラウド・ファンディング（crowdfunding）」という言葉は、最近使われ始めた新しい造語です。  
また、時代によって言葉の意味が変化するケースもあります。  
たとえば、「heavy」には、「（ものごとが）深刻である」という意味がありますが、昔はそのような使い方はされませんでした。  
このような単語の変化に対応するには、シソーラスを人手によって絶えず更新しなければなりません。  

**人の作業コストが高い**  
シソーラスを作るには、大変な人的コストが発生します。  
英語を例に挙げると、現存する英単語の総数は 1,000 万語を超えると言われています。  
そのため、理想的にはそのような膨大な単語に対して、単語の関連付けを行う必要があるのです。  
ちなみに WordNet では、20 万語を超える単語が登録されているそうです。

**単語の細かなニュアンスを表現できない**  
シソーラスでは、類義語として似たような単語をグループ化します。  
しかし実際には、似たような単語であっても、それぞれにニュアンスは異なるものです。  
たとえば、「ヴィンテージ」という単語と「レトロ」という単語は同じような意味を表しますが、その使われ方は異なります。  
シソーラスでは、このような微妙なニュアンスの差異を表すことができません（もし人手で表すとすれば、それは相当に困難なものになるでしょう）。  
<br>

このように、シソーラスを使う手法には大きな問題があります。   
このような問題を避けるために「カウントベースの手法」、ニューラルネットワークを使った「推論ベースの手法」へと進みます。  
それら２つの手法では、大量のテキストデータから自動的に「単語の意味」を抽出します。  
そのため、人の手作業によって単語を関連付けるというような重労働から解放されるのです！
</div>

<div style="">

![代替テキスト](./kazuki.a_images/Section2/Hint/Hint2-2_3.png)
</div>

<div style="color: red;">

## **2.3 カウントベースの手法**
</div>

<div style="color: black; font-size: 20px">

これからカウントベースの手法に進むにあたって、私たちは **コーパス**（corpus）を利用します。  
**コーパス** とは、簡単に言ってしまえば、大量のテキストデータです。  
ただし、やみくもに集められたテキストデータではなく、  
自然言語処理の研究やアプリケーションのために目的をもって収集されたテキストデータを、一般的には「コーパス」と呼びます。

結局のところ、コーパスとはテキストデータにすぎません。  
しかし、そこに含まれる文章は人の手によって書かれたものです。  
これは別の見方をすれば、コーパスには自然言語に対する人の "知識" がふんだんに含まれているということです。  
文章の書き方、単語の選び方、そして、単語の意味――そのような人の自然言語に対する知識が、コーパスには含まれています。  
カウントベースの手法の目標は、そのような人の知識が詰まったコーパスから、自動的に、そして効率良く、そのエッセンスを抽出することにあります。

![代替テキスト](./kazuki.a_images/Section2/Caution/Caution2-3_1.png)
</div>

<div style="color: red;">

### **2.3.1 Python によるコーパスの下準備**
</div>

<div style="color: black; font-size: 20px">

自然言語処理の分野で用いられるコーパスには、さまざまなものが存在します。  
有名なもので言えば、Wikipedia や Google News などのテキストデータが挙げられます。  
また、シェイクスピアや夏目漱石など、偉大な作家の作品群もコーパスとして利用されます。  
本章では、まず初めに１文からなる単純なテキストをコーパスとして利用します。  
その後で、より実用的なコーパスを扱います。  
それでは Python の対話モードを使って、とても小さなテキストデータ（コーパス）に前処理を行います。  
ここで言う前処理とは、テキストデータを単語に分割し、その分割した単語を単語 ID のリストへと変換することです。  
では、ひとつずつ確認しながら、ステップ・バイ・ステップで組み立てていきましょう。  
まずは、今回コーパスとして利用するサンプルの文章からです。
</div>

In [1]:
text = 'You say goodbye and I say hellow.'

<div style="color: black; font-size: 20px">

ここではひとつの文からなるテキストをコーパスとして利用します。  
本来であれば、このテキスト（<span style="font-family: Consolas">text</span>）には何千、何万を超える文が（連結されて）含まれます。  
しかし、ここでは簡易性を優先して、この小さなテキストデータに対して前処理を行っていきます。  
それでは、上の <span style="font-family: Consolas">text</span> を単語単位に分割しましょう。
</div>

In [2]:
text = text.lower()
text = text.replace('.', ' .')
text

'you say goodbye and i say hellow .'

In [3]:
words = text.split(' ')
words

['you', 'say', 'goodbye', 'and', 'i', 'say', 'hellow', '.']

<div style="color: black; font-size: 20px">

ここでは最初に、すべての文字を小文字に変換するために <span style="font-family: Consolas">lower()</span> メソッドを使います。  
これは、文頭の単語でも共通の単語として扱えるようにするための処置です。  
そして、<span style="font-family: Consolas">split(' ')</span> によって、スペースを「区切り文字」として分割します。  
ただしここでは文末のピリオド (<span style="font-family: Consolas">.</span>) を考慮して、ピリオドの前にスペースを挿入してから分割を行っています。

![代替テキスト](./kazuki.a_images/Section2/Caution/Caution2-3_2.png)
</div>

<div style="color: black; font-size: 20px">

これで、元の文章を単語のリストとして利用できるようになりました。  
単語が分割されたことで扱いやすくなりましたが、単語をテキストのまま操作するのは何かと不便です。  
そこで、単語に ID を振って、ID のリストとして利用できるように、さらに手を加えます。  
その下準備として、単語の ID と単語の対応表を Python のディクショナリで作成します。
</div>

In [5]:
word_to_id = {}
id_to_word = {}
for word in words:
    if word not in word_to_id:
        new_id = len(word_to_id)
        word_to_id[word] = new_id
        id_to_word[new_id] = word

<div style="color: black; font-size: 20px">

単語 ID → 単語 の変換を <span style="font-family: Consolas">id_to_word</span> という変数が担当し（キー: 単語ID、値: 単語）、  
単語 → 単語 ID の変換を <span style="font-family: Consolas">word_to_id</span> が担当します。  

ここでは単語単位に分割された <span style="font-family: Consolas">words</span> の各要素を先頭からひとつずつ見ていき、  
単語が <span style="font-family: Consolas">word_to_id</span> に存在しない場合は、<span style="font-family: Consolas">word_to_id</span> と <span style="font-family: Consolas">id_to_word</span> に新しい ID と単語をそれぞれ追加します。  

なお、ディクショナリの長さを新しい単語 ID として設定するため、単語 ID は、0、1、2……と増えていくことになります。  
これで、単語 ID と単語の対応表を作ることができました。それでは、実際に中身を見てみましょう。
</div>

In [8]:
id_to_word

{0: 'you', 1: 'say', 2: 'goodbye', 3: 'and', 4: 'i', 5: 'hellow', 6: '.'}

In [7]:
word_to_id

{'you': 0, 'say': 1, 'goodbye': 2, 'and': 3, 'i': 4, 'hellow': 5, '.': 6}

<div style="color: black; font-size: 20px">

それでは最後に、「単語のリスト」を「単語ID のリスト」に変換しましょう。  
ここでは、Python の内包表記を使って、単語リストから単語ID リストへ変換します。  
そして、それをNumPy 配列に変換します。
</div>

In [9]:
import numpy as np
corpus = [word_to_id[w] for w in words]
corpus = np.array(corpus)
corpus

array([0, 1, 2, 3, 4, 1, 5, 6])

![代替テキスト](./kazuki.a_images/Section2/Hint/Hint2-3_1.png)

<div style="color: black; font-size: 20px">

これで、コーパスを利用する下準備が整いました。  
それでは以上の処理を <span style="font-family: Consolas">preprocess()</span> という名前の関数として、まとめて実装することにします。
</div>

In [10]:
def preprocess(text):
    text = text.lower()
    text = text.replace('.', ' .')
    words = text.split(' ')

    word_to_id = {}
    id_to_word = {}
    for word in words:
        if word not in word_to_id:
            new_id = len(word_to_id)
            word_to_id[word] = new_id
            id_to_word[new_id] = word

    corpus = np.array([word_to_id[w] for w in words])

    return corpus, word_to_id, id_to_word

<div style="color: black; font-size: 20px">

この関数を使えば、コーパスの前処理は次のように行うことができます。
</div>

In [12]:
text = 'You say goodbye and I say hello.'
corpus, word_to_id, id_to_word = preprocess(text)

<div style="color: black; font-size: 20px">

これでコーパスの前処理は終わりです。  
ここで準備した <span style="font-family: Consolas">corpus</span>、<span style="font-family: Consolas">word_to_id</span>、<span style="font-family: Consolas">id_to_word</span> という３つの変数名は、これから先も本書の多くの場所で登場します。  
<span style="font-family: Consolas">corpus</span> は単語 ID のリスト、<span style="font-family: Consolas">word_to_id</span> は単語から単語 ID へのディクショナリ、<span style="font-family: Consolas">id_to_word</span> は単語 ID から単語へのディクショナリを表します。  
以上でコーパスを扱う準備が整いました。  

私たちのこれからの目標は、コーパスを使って「単語の意味」を抽出することです。  
そのために本節では「カウントベースの手法」を見ていきます。  
この手法によって、私たちは単語をベクトルで表すことができるようになるのです。
</div>

<div style="color: red;">

### **2.3.2 単語の分散表現**
</div>

<div style="color: black; font-size: 20px">

突然ですが、世の中にはさまざまな「色」が存在します。  
たとえば、「コバルトブルー」や「シンクレッド」といったように、色には固有の名前が付けられています。  
その一方で、色は RGB（Red/Green/Blue）の３成分がどれだけ存在するかといった方法でも表すことができます。  

前者が色の数だけ異なる名前を命名する一方で、後者は、色を**３次元のベクトルとして表現**します。  
ここで注目したい点は、RGB のようなベクトル表現のほうが、正確に色を指定できるということです。  
さらに、３つの成分というコンパクトな表現が可能であり、（多くの場合）色のイメージがつきやすいという利点もあります。  

たとえば、「こきあけ深緋」という色がどのような色か分からなくても、(R, G, B) = (201, 23, 30) と言われれば、それは赤系の色だということが分かります。  
また、色どうしの関連性 ――似た色かどうかなど―― もベクトル表現のほうが容易に判断しやすく、定量化も簡単に行えます。  

それでは、ここで話した「色」のベクトル表現のようなことを、「単語」でも行えないでしょうか？  
より正確に言うならば、コンパクトで理にかなったベクトル表現を、「単語」というフィールドにおいても構築できないでしょうか？  

これから私たちが目指すべき場所は、「単語の意味」を的確に捉えた**ベクトル表現**です。  
これは自然言語処理の分野では、単語の**分散表現**と呼ばれます。

![代替テキスト](./kazuki.a_images/Section2/Hint/Hint2-3_2.png)
</div>

<div style="color: red;">

### **2.3.3 分布仮説**
</div>

<div style="color: black; font-size: 20px">

自然言語処理の歴史において、単語をベクトルで表す研究は数多く行われてきました。  
そのような研究を見ていくと、重要な手法のほとんどすべてが、あるひとつのシンプルなアイデアに基づいていることが分かります。  

そのアイデアとは、「**単語の意味は、周囲の単語によって形成される**」というものです。  
これは、**分布仮説**（distributional hypothesis）と呼ばれるもので、単語をベクトルで表す最近の研究の多くが、この仮説に基づいています。

分布仮説が言わんとすることは、とてもシンプルです。  
単語自体には意味がなく、その単語の「コンテキスト（文脈）」によって、単語の意味が形成されると言うのです。  
確かに、意味的に同じ単語は、同じような文脈で多く出現します。  
たとえば、「I drink beer.」「We drink wine.」のように drink の近くには飲み物が表れやすいでしょう。  
また、「I guzzle beer.」「We guzzle wine.」のような文章があれば、guzzle という単語が drink と同じような文脈で使われるということが分かります。  
そして、guzzle と drink が近い意味の単語だということも導けます（ちなみに、guzzle とは「がぶがぶ飲む」という意味の単語です）。

さて、これから先、「コンテキスト」という言葉を多く使います。  
本章で「コンテキスト」と言うとき、それは（注目する単語に対して）その周囲に存在する単語を指します。  
たとえば、図2-3 の例では、左右の２単語が「コンテキスト」に相当します。

![代替テキスト](./kazuki.a_images/Section2/Figure/図2-3_コンテキスト.png)

図2-3 に示すとおり、「コンテキスト」とは、ある中央の単語に対して、その周囲にある単語を指します。  
またここでは、コンテキストのサイズ――つまり、周囲の単語をどれだけ含めるかということ――を、「ウィンドウサイズ（window size）」という言葉で表すことにします。  
ウィンドウサイズが１の場合は左右の１単語、ウィンドウサイズが２の場合は左右の２単語、といったようになります。
</div>

<div style="color: black; font-size: 20px">

![代替テキスト](./kazuki.a_images/Section2/Caution/Caution2-3_3.png)
</div>

<div style="color: red;">

### **2.3.4 共起行列**
</div>

<div style="color: black; font-size: 20px">

それでは、分布仮説に基づいて、単語をベクトルで表す方法を考えましょう。  
そのための素直な方法は、周囲の単語を "カウント" することです。  

具体的に言うと、ある単語に着目した場合、その周囲にどのような単語がどれだけ現れるのかをカウントし、集計するのです。  
ここではこれを「カウントベースの手法」と呼ぶことにします。  
なお、これは文献によっては「統計的手法」と呼ばれる場合もあります。  

それでは、カウントベースの手法を見ていきましょう。  
ここでは、「2.3.1 Python によるコーパスの下準備」のコーパスと <span style="font-family: Consolas">preprocess()</span> 関数を使用して、再度下準備を行うところからスタートします。
</div>

In [13]:
import sys
sys.path.append('..')
import numpy as np
from common.util import preprocess

text = 'You say goodbye and I say hello.'
corpus, word_to_id, id_to_word = preprocess(text)
print(corpus)
print(id_to_word)

[0 1 2 3 4 1 5 6]
{0: 'you', 1: 'say', 2: 'goodbye', 3: 'and', 4: 'i', 5: 'hello', 6: '.'}


<div style="color: black; font-size: 20px">

上の結果より、ここでは語彙数が全部で７個あることが分かります。  
それでは、それぞれの単語について、そのコンテキストに含まれる単語の頻度を数えていきます。  
ここでは、ウィンドウサイズを 1 として、まずは単語ID が 0 の「you」からスタートします。

![代替テキスト](./kazuki.a_images/Section2/Figure/図2-4_youのコンテキストカウント.png)

図2-4 を見れば一目瞭然ですが、単語「you」のコンテキストには「say」という単語がひとつだけ存在することが分かります。  

これをテーブルで表せば、図2-5 のようになります。

![代替テキスト](./kazuki.a_images/Section2/Figure/図2-5_youのコンテキストカウントテーブル.png)

図2-5 は、単語「you」に対して、コンテキストとして共起する単語の頻度を表しています。  
そしてこれは同時に「you」という単語が <span style="font-family: Consolas">[0, 1, 0, 0, 0, 0, 0]</span> というベクトルで表現できるということも意味します。
</div>

<div style="color: black; font-size: 20px">

それでは、続いて単語 ID が 1 の「say」について、同じ作業を行います。  

その結果は、図2-6 のようになります。

![代替テキスト](./kazuki.a_images/Section2/Figure/図2-6_sayのコンテキストカウントテーブル.png)

上の結果から、「say」という単語は <span style="font-family: Consolas">[1, 0, 1, 0, 1, 1, 0]</span> というベクトルで表現できることが分かります。  
以上の作業を、すべての単語について行います。その結果は、図2-7 のようになります。

![代替テキスト](./kazuki.a_images/Section2/Figure/図2-7_各単語のコンテキストカウントテーブル.png)
</div>

<div style="color: black; font-size: 20px">

図2-7 は、すべての単語に対して、共起する単語をテーブルにまとめたものです。  
このテーブルの各行が、該当する単語のベクトルに対応します。  
なお、図2-7 のテーブルは行列の形をしているため、**共起行列**（co-occurrence matrix）と呼ばれます。  
それでは、上の共起行列を実際に作ってみましょう。  
ここでは、図2-7 の結果をそのまま手で打ち込むことにします。
</div>

In [ ]:
C = np.array([
    [0, 1, 0, 0, 0, 0, 0],
    [1, 0, 1, 0, 1, 1, 0],
    [0, 1, 0, 1, 0, 0, 0],
    [0, 0, 1, 0, 1, 0, 0],
    [0, 1, 0, 1, 0, 0, 0],
    [0, 1, 0, 0, 0, 0, 1],
    [0, 0, 0, 0, 0, 1, 0],
], dtype=np.int32)

<div style="color: black; font-size: 20px">

これで共起行列ができました。この共起行列を使えば、各単語のベクトルは次のようにして得ることができます。
</div>

In [20]:
print("単語ID が 0 のベクトル:", C[0])
print("単語ID が 4 のベクトル:", C[4])
print("'goodbye' のベクトル :", C[word_to_id['goodbye']])

単語ID が 0 のベクトル: [0 1 0 0 0 0 0]
単語ID が 4 のベクトル: [0 1 0 1 0 0 0]
'goodbye' のベクトル : [0 1 0 1 0 0 0]


<div style="color: black; font-size: 20px">

ここで示したように、私たちは共起行列によって単語をベクトルで表すことができました。  
ここでは手動で共起行列を作りましたが、もちろんこれは自動化できます。  
それでは、コーパスから共起行列を作る関数を実装しましょう。  

ここでは、<span style="font-family: Consolas">create_co_matrix(corpus, vocab_size, window_size=1)</span> という関数名で実装します。  
引数の <span style="font-family: Consolas">corpus</span> は単語 ID のリスト、<span style="font-family: Consolas">vocab_size</span> は語彙数、<span style="font-family: Consolas">window_size</span> はウィンドウサイズを表します。

In [21]:
def create_co_matrix(corpus, vocab_size, window_size=1):
    corpus_size = len(corpus)
    co_matrix = np.zeros((vocab_size, vocab_size), dtype=np.int32)

    for idx, word_id in enumerate(corpus):
        for i in range(1, window_size + 1):
            left_idx = idx - i
            right_idx = idx + i

            if left_idx >= 0:
                left_word_id = corpus[left_idx]
                co_matrix[word_id, left_word_id] += 1

            if right_idx < corpus_size:
                right_word_id = corpus[right_idx]
                co_matrix[word_id, right_word_id] += 1

    return co_matrix

<div style="color: black; font-size: 20px">

ここでは、まず初めに <span style="font-family: Consolas">co_matrix</span> を要素が 0 の２次元配列で初期化します。  
後は、コーパス中の各単語すべてに対して、そのウィンドウに含まれる単語をカウントしていきます。  
このとき、コーパスの左端と右端において、はみ出していないかのチェックを行います。  
これでコーパスがどれだけ大きくなったとしても、自動で共起行列を作ることができます。  
これ以降、私たちはこの関数を使ってコーパスの共起行列を作成します。
</div>